In [ ]:
# import necessary packages
from kafka import KafkaConsumer
import json
import logging
import time
import threading
from kafka.structs import TopicPartition

In [ ]:
# function to retrieve messages from Kafka Server for a given topic & partition
GroupID = "Enter Group ID"
def Consume_messages(my_topic,my_partition):
    try:
        print('thread started for topic: '+my_topic+ ' & partition: '+str(my_partition))
        consumer = KafkaConsumer(bootstrap_servers=bootstrap_servers, group_id=GroupID,
            consumer_timeout_ms=2000,
            value_deserializer=lambda x: json.loads(x.decode('utf-8')))
        tp = TopicPartition(topic=my_topic, partition=my_partition)
        consumer.assign([tp])
        while True:
            txt = {}
            msg_count = 0
            while msg_count <= 50:
                for msg in consumer:
                    consumer.commit()
                    txt.update(msg.value)
                    msg_count = msg_count + 1
                    print(msg.value)
                    time.sleep(0.5)
            stock_dataframe = pd.DataFrame.from_records(txt).T
            #Save the data into CSV Format
            stock_dataframe.to_csv('Rt_data.csv')
            time.sleep(15)
    except Exception as e:
            logging.warning('exception while pulling data from Kafka')

In [ ]:
# give name to Kafka Topic & define local Kafka server
topic = 'Enter Topic Name'
bootstrap_servers = 'localhost:9092'

In [ ]:
consumer = KafkaConsumer(bootstrap_servers=bootstrap_servers, group_id=GroupID,
    consumer_timeout_ms=2000,
    value_deserializer=lambda x: json.loads(x.decode('utf-8')))
partitions=consumer.partitions_for_topic(topic=topic)
threads = []
for p in partitions:
    t = time.time()
    # create the thread
    threads.append(threading.Thread(target=Consume_messages, args=(topic,p,)))
    print (topic + ' -> '+str(p))
    threads[-1].start() # start the thread we just created
# wait for all threads to finish                                            
for t in threads:                                                           
    t.join() 

In [ ]:
# close the Kafka consumer object
consumer.close()